In [1]:
import tweepy
import csv
import json
from config import (consumer_key, consumer_secret,
                    access_token, access_token_secret)
import wget
from bson import ObjectId # For ObjectId to work
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
import numpy as np
import time


In [2]:
uri = "mongodb://newsdata:ZG4cTh2XbyUpkiVggEcQ6lL6UEC32iJThy4eZrO9RLY1suKiXdUNrqi5pzrqzqNvUsdO9xAxjhyFfNfNVoBeXA==@newsdata.documents.azure.com:10255/?ssl=true&replicaSet=globaldb"
client = MongoClient(uri)
db = client.fakenewsdb
politico_facts_data_table = db.politico_facts_data
print(politico_facts_data_table.find_one())

{'_id': ObjectId('5c23d50044925f3eb8d1a2ef'), 'Date': 'on Sunday, December 23rd, 2018', 'Meter': 'Half-True', 'Source': 'Garry McCarthy', 'Statement': '"There was almost a 70 percent reduction in police-related shootings under my tenure because of training, supervision and policy decisions."\r\n', 'Statement_url': '/illinois/statements/2018/dec/23/garry-mccarthy/did-police-shootings-plunge-under-mccarthys-leader/', 'Fact': "The numbers fell, but that doesn't mean he can take all the credit", 'Source_Url': '/personalities/garry-mccarthy/statements/byruling/half-true/'}


In [4]:
records = db.twitter_data_combined.find().sort([('_id', -1)]).limit(1)
for r in records:
    print(r)

{'_id': ObjectId('5c359c07f7421b47b0d92757'), 'id': 'politifact13589', 'title': 'Pope Francis To Followers: “Koran And Holy Bible Are The Same”', 'twitter_text': 'Pope Francis To Followers: "Koran And Holy Bible Are The Same" | National Report https://t.co/686RVPQZF6 via @wpusta', 'twitter_location': 'DENMARK', 'twitter_time': '2016-02-21 20:01:10', 'twitter_user': 'PRINTSVONJANE'}


In [4]:
#fakenews_items = fakenews_datatable.find({}).limit(20);

#fakenews_datatable.aggregate( [ { "$unwind": "$tags" },  { "count": "$tweets" } ] ).limit(10)

items = fakenews_datatable.aggregate([{
  "$project": {
      "_id": "$_id",
      "id":"$id",
      "text_content":"$text_content.$text",
      "tweetCount": { "$size": "$tweets" }
  }
    
}]);

df = pd.DataFrame(list(items))
df.head()

,_id,id,tweetCount
0,5c3445ffe8a490580c567e3c,politifact14927,197
1,5c3445ffe8a490580c567e3d,politifact14856,116
2,5c3445ffe8a490580c567e3e,politifact14733,33
3,5c3445ffe8a490580c567e3f,politifact14693,4
4,5c3445ffe8a490580c567e40,politifact14362,25


In [5]:
df = df.sort_values(by='tweetCount',ascending=False)

In [6]:
df.head(10)

,_id,id,tweetCount
84,5c34465ae8a490580c567e7b,politifact15018,1141
9,5c3445ffe8a490580c567e46,politifact14259,975
35,5c34461499023e6294754497,politifact14839,963
63,5c34463de8a490580c567e71,politifact13589,827
38,5c34462699023e62947544ac,politifact15004,430
30,5c344615f839832ff86827f0,politifact13854,376
19,5c34461144925f59105fd04f,politifact14238,352
102,5c34466599023e62947544bb,politifact15334,333
5,5c3445ffe8a490580c567e41,politifact15201,329
103,5c34466599023e62947544bc,politifact15048,296


In [7]:
df2=df.head(10)
df2['news_title']=""
for index,row in df2.iterrows():
    df2.set_value(index,'news_title',fakenews_datatable.find_one({"id":row["id"]})['text_content']['title'])

df2.head()

C:\Users\chami\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\chami\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


,_id,id,tweetCount,news_title
84,5c34465ae8a490580c567e7b,politifact15018,1141,Smoking In Cars With Children Is Illegal Start...
9,5c3445ffe8a490580c567e46,politifact14259,975,Bumble Bee Foods charged after man cooked with...
35,5c34461499023e6294754497,politifact14839,963,Morgue employee cremated by mistake while taki...
63,5c34463de8a490580c567e71,politifact13589,827,Pope Francis To Followers: “Koran And Holy Bib...
38,5c34462699023e62947544ac,politifact15004,430,"NFL Lawyer, Who Claimed Super Bowl Is ‘Rigged’..."


In [7]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [10]:
twitter_data_list = []

politifacts=db.fakenews_data.find_one({"id":"politifact15004"})
print(politifacts['text_content']['title'])
    
for tweet in politifacts['tweets']:
        twitter_data_dic = {}
        try:
            twitter_data_dic['id']="politifact15004"
            twitter_data_dic['title']=politifacts['text_content']['title']
            
            twitter_data=api.get_status(tweet['tweet_id'])
            print(twitter_data.text)
            print(twitter_data.user.location)
            print(datetime.utcfromtimestamp(tweet['created_at']).strftime('%Y-%m-%d %H:%M:%S'))
            print(twitter_data.user.screen_name)
            twitter_data_dic['twitter_text']=twitter_data.text
            twitter_data_dic['twitter_location']=twitter_data.user.location
            twitter_data_dic['twitter_time']=datetime.utcfromtimestamp(tweet['created_at']).strftime('%Y-%m-%d %H:%M:%S')
            twitter_data_dic['twitter_user']=twitter_data.user.screen_name
            
            twitter_data_list.append(twitter_data_dic)
            db.twitter_data_combined.insert(twitter_data_dic)
       
        except tweepy.RateLimitError:
            print("rate limit exceeded")
            time.sleep(60 * 15)
            continue
        

    
    

NFL Lawyer, Who Claimed Super Bowl Is ‘Rigged’, Found Dead


TweepError: [{'code': 63, 'message': 'User has been suspended.'}]

In [ ]:
combined_data = pd.DataFrame(twitter_data_list)
combined_data

In [ ]:
combined_data.to_json(orient='records')

In [ ]:
with open('data.json', 'w') as outfile:
    json.dump(combined_data.to_json(orient='records'), outfile)

In [ ]:
combined_data.to_csv("data.csv")